In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
class BitcoinPricePredictor:
    def __init__(self, lookback=24, future_steps=1):
        self.lookback = lookback
        self.future_steps = future_steps
        self.scaler = MinMaxScaler()
        self.model = None
        
    def prepare_data(self, data, train_split=0.8):
        # Assume data is a pandas DataFrame with 'price' column
        scaled_data = self.scaler.fit_transform(data[['price']].values)
        
        X, y = [], []
        for i in range(len(scaled_data) - self.lookback - self.future_steps + 1):
            X.append(scaled_data[i:(i + self.lookback)])
            y.append(scaled_data[i + self.lookback:i + self.lookback + self.future_steps])
        
        X = np.array(X)
        y = np.array(y)
        
        # Split into train and test sets
        train_size = int(len(X) * train_split)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]
        
        return X_train, X_test, y_train, y_test
    
    def build_model(self):
        model = Sequential([
            # First BiLSTM layer
            Bidirectional(LSTM(128, return_sequences=True), 
                         input_shape=(self.lookback, 1)),
            BatchNormalization(),
            Dropout(0.3),
            
            # Second BiLSTM layer
            Bidirectional(LSTM(256, return_sequences=True)),
            BatchNormalization(),
            Dropout(0.4),
            
            # Third BiLSTM layer
            Bidirectional(LSTM(128, return_sequences=False)),
            BatchNormalization(),
            Dropout(0.3),
            
            # Dense layers
            Dense(64, activation='relu'),
            BatchNormalization(),
            Dropout(0.2),
            
            Dense(32, activation='relu'),
            BatchNormalization(),
            
            Dense(self.future_steps)
        ])
        
        optimizer = Adam(learning_rate=0.001)
        model.compile(optimizer=optimizer, loss='mse')
        self.model = model
        return model
    
    def train(self, X_train, y_train, X_val, y_val, epochs=100, batch_size=32):
        callbacks = [
            EarlyStopping(patience=10, restore_best_weights=True),
            ReduceLROnPlateau(factor=0.5, patience=5, min_lr=0.00001)
        ]
        
        history = self.model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks,
            verbose=1
        )
        return history
    
    def predict(self, X):
        predictions = self.model.predict(X)
        return self.scaler.inverse_transform(predictions)
    
    def evaluate(self, y_true, y_pred):
        mse = mean_squared_error(y_true, y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        return {
            'MSE': mse,
            'RMSE': np.sqrt(mse),
            'MAE': mae,
            'R2': r2
        }
    
    def plot_predictions(self, y_true, y_pred, title="Bitcoin Price Prediction"):
        plt.figure(figsize=(15, 6))
        plt.plot(y_true, label='Actual Price', color='blue')
        plt.plot(y_pred, label='Predicted Price', color='red')
        plt.title(title)
        plt.xlabel('Time')
        plt.ylabel('Price (USD)')
        plt.legend()
        plt.grid(True)
        plt.show()

In [ ]:


# Example usage
def main():
    # Assuming you have a DataFrame 'df' with columns including 'price'
    # df = pd.read_csv('your_bitcoin_data.csv')
    
    # Create synthetic data for demonstration
    np.random.seed(42)
    dates = pd.date_range(start='2020-01-01', periods=38000, freq='H')
    prices = np.random.normal(loc=40000, scale=5000, size=38000)
    prices = np.cumsum(np.random.normal(loc=0, scale=100, size=38000)) + prices
    df = pd.DataFrame({'timestamp': dates, 'price': prices})
    
    # Initialize predictor
    predictor = BitcoinPricePredictor(lookback=24, future_steps=1)
    
    # Prepare data
    X_train, X_test, y_train, y_test = predictor.prepare_data(df)
    
    # Build and train model
    predictor.build_model()
    predictor.model.summary()
    
    # Split training data into train and validation
    val_split = int(0.9 * len(X_train))
    X_train_final = X_train[:val_split]
    y_train_final = y_train[:val_split]
    X_val = X_train[val_split:]
    y_val = y_train[val_split:]
    
    # Train model
    history = predictor.train(X_train_final, y_train_final, X_val, y_val)
    
    # Make predictions
    train_predictions = predictor.predict(X_train)
    test_predictions = predictor.predict(X_test)
    
    # Inverse transform actual values
    y_train_actual = predictor.scaler.inverse_transform(y_train)
    y_test_actual = predictor.scaler.inverse_transform(y_test)
    
    # Evaluate model
    train_metrics = predictor.evaluate(y_train_actual, train_predictions)
    test_metrics = predictor.evaluate(y_test_actual, test_predictions)
    
    print("\nTraining Metrics:")
    for metric, value in train_metrics.items():
        print(f"{metric}: {value:.4f}")
    
    print("\nTest Metrics:")
    for metric, value in test_metrics.items():
        print(f"{metric}: {value:.4f}")
    
    # Plot results
    predictor.plot_predictions(y_test_actual, test_predictions, "Bitcoin Price Prediction - Test Set")


In [ ]:

if __name__ == "__main__":
    main()